<h1> Create ET weater psql database  </h1>

In [24]:
import pandas as pd
import psycopg2
from os import listdir
from collections import OrderedDict
from ethiopian_date import ethiopian_date

<h3> constants </h3>

In [139]:
COLUMN_TYPES = OrderedDict([('date', 'date'), ('e_date', 'date'), ('GeoKey', 'text'), \
            ('precipitation', 'float'), ('humid_max', 'float'), ('humid_min', 'float'), ('solar', 'float'), 
            ('temp_max', 'float'), ('temp_min', 'float'), ('wind_avg', 'float')])

KEYS = COLUMN_TYPES.keys()
KEY_STR = ', '.join(KEYS)
VAL_STR = ' ,'.join(['%s'] * len(KEYS))
DATE_FORMAT = '%Y-%m-%d'

<h3> functions </h3>

In [140]:
def make_edate(date):
    '''If 13th month we move forward 6 days to avoid error. The 13th month is lumped in with the 1st month'''
    try:
        return ethiopian_date.EthiopianDateConverter.date_to_ethiopian(date)
    except ValueError:
        return ethiopian_date.EthiopianDateConverter.date_to_ethiopian(date + pd.offsets.relativedelta(days=6))
    
def write_to_psql(row):
    #insert into postgres
    cur.execute('INSERT INTO daily ({:s}) VALUES ({:s})'.format(KEY_STR, VAL_STR), row[KEYS].values)
    conn.commit() #submit change to db

<h3> Establish connections </h3>

In [177]:
conn = psycopg2.connect("host=localhost port=5432 dbname=et_weather user=attiladobi")
cur = conn.cursor()

<h1> Create new table for daily weather</h1>

In [173]:
column_str = ', '.join(['%s %s' % column for column in COLUMN_TYPES.items()])
cur.execute('CREATE TABLE daily (id serial PRIMARY KEY, %s);' % column_str)
conn.commit() #submit change to db

<h1> Loop through all the woreda weather files and write to postgres db </h1>

In [174]:
weather_dir = '/Users/attiladobi/PyNotebooks/zenysis/weather'
for filename in [csvfile for csvfile in listdir(weather_dir) if '.csv' in csvfile]:
    weather_df = pd.read_csv('%s/%s' % (weather_dir, filename), index_col=0)
    weather_df['e_date'] = [make_edate(date).strftime(DATE_FORMAT) for date in pd.to_datetime(weather_df['date'])]
    # Write NaN as NULL in postgres to avoid issues with aggregation
    weather_df = weather_df.where(pd.notnull(weather_df), None)
    # Loop through each row writing to postgres
    for row in weather_df.iterrows():
        write_to_psql(row[1])


<h1> Create a new monthly table </h1>

In [178]:
# runs in about 10s
create_mothly_str = \
"CREATE TABLE monthly AS \
SELECT date_trunc('month', e_date) AS e_date, \
geokey, \
SUM(precipitation) as precipitation, \
MAX(precipitation) as max_precipitation, \
SUM(CASE WHEN precipitation = 0 THEN 0 ELSE 1 END) as days_precipitation, \
AVG(wind_avg) as wind_avg, \
MAX(temp_max) as temp_max, \
MIN(temp_min) as temp_min, \
AVG((temp_min + temp_max)/2) as temp_avg, \
MAX(humid_max) as humid_max, \
MIN(humid_min) as humid_min, \
AVG((humid_max + humid_min)/2) as humid_avg, \
AVG(solar) as solar \
from daily group by (geokey, 1);"

cur.execute(create_mothly_str)
conn.commit()

In [179]:
conn.close()

<h1> Add functions to take updates, tack on to the daily db and update momthly table </h1>